In [169]:
import pyodbc 
import pandas as pd
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=VisionDB;'
                      'Database=Sandbox_Vision;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

In [170]:
query = """ SELECT VE.Name
      ,VEA.[Vendor]
      ,VEA.[Address]
      ,VEA.[Address1]
      ,VEA.[Address2]
      ,VEA.[Address3]
      ,VEA.[Address4]
      ,VEA.[City]
      ,VEA.[State]
      ,VEA.[ZIP]
  FROM [Sandbox_Vision].[dbo].[VEAddress] VEA
  inner join [Sandbox_Vision].[dbo].VE 
  ON VEA.Vendor=VE.Vendor
  WHERE VE.LinkedClient IS NULL
  """

In [171]:
Vendors = pd.read_sql_query(query,conn)

In [172]:
#number of vendor records
len(Vendors)

16667

In [173]:
#get rid of dupes
Vendors=Vendors.drop_duplicates(subset=['Vendor'])

In [174]:
query = """ 
SELECT        
    CL.Name, 
    CLAddress.Address as AddressProfileName, 
    CLAddress.Address1, CLAddress.Address2, CLAddress.Address3, 
    CLAddress.Address4, CLAddress.City, CLAddress.ZIP, CLAddress.State,CL.Status,CL.ClientID                        
FROM            
    CLAddress 
    INNER JOIN CL ON CLAddress.ClientID = CL.ClientID
    """

In [175]:
Clients = pd.read_sql_query(query,conn)

In [176]:
#clients count
len(Clients)

46806

In [177]:
import nltk

In [178]:
#with this we do not swap strings, but instead we swap tokens, streets does not become sts, it remains streets and 
#street will become st

def swaptoken(tokenlist,toswap,swapwith):
    returnlist=[]
    for i in tokenlist:
        if i==toswap:
            returnlist.append(swapwith)
        else:
            returnlist.append(i)
    return (returnlist)

In [179]:
#drive -> dr
#street -> st
#avenue -> ave
#boulevard->blvd
#road -> rd
#north -> n
#south -> s
#west -> w
#east -> e
#highway -> hwy
#llc -> ''

In [180]:
Vendors['Address12']=Vendors['Address1'].str.lower().str.replace('.','').str.replace(',','').str.replace('-','').str.replace('#','').\
    astype(str).apply(nltk.word_tokenize).\
    apply(swaptoken, args=('st','street')).apply(swaptoken, args=('dr','drive')).\
    apply(swaptoken, args=('ave','avenue')).apply(swaptoken, args=('blvd','boulevard')).\
    apply(swaptoken, args=('rd','road')).\
    apply(swaptoken, args=('n','north')).apply(swaptoken, args=('s','south')).\
    apply(swaptoken, args=('w','west')).apply(swaptoken, args=('e','east')).\
    apply(swaptoken, args=('hwy','highway')).apply(swaptoken, args=('llc','')).apply(' '.join)

In [181]:
Clients['Address12']=Clients['Address1'].str.lower().str.replace('.','').str.replace(',','').str.replace('-','').str.replace('#','').\
    astype(str).apply(nltk.word_tokenize).\
    apply(swaptoken, args=('st','street')).apply(swaptoken, args=('dr','drive')).\
    apply(swaptoken, args=('ave','avenue')).apply(swaptoken, args=('blvd','boulevard')).\
    apply(swaptoken, args=('rd','road')).\
    apply(swaptoken, args=('n','north')).apply(swaptoken, args=('s','south')).\
    apply(swaptoken, args=('w','west')).apply(swaptoken, args=('e','east')).\
    apply(swaptoken, args=('hwy','highway')).apply(swaptoken, args=('llc','')).apply(' '.join)

In [182]:
#get rid of duplicate clients
Clients=Clients.drop_duplicates(subset=['ClientID'])

In [183]:
Clients['CleanedAddress']=(Clients['Name']+" "+Clients['Address12']+" "+Clients['City']+" "+Clients['State']+" "+Clients['ZIP']).\
        str.replace(',','').str.replace('.','').str.replace('-',' ').str.lower()

In [184]:
Vendors['CleanedAddress']=(Vendors['Name']+" "+Vendors['Address12']+" "+Vendors['City']+" "+Vendors['State']+" "+Vendors['ZIP']).\
        str.replace(',','').str.replace('.','').str.replace('-',' ').str.lower()

In [185]:
Vendors2=Vendors[Vendors['Name'].str.lower().isin(Clients['Name'].str.lower().to_list())==False]

In [186]:
#number of vendors with names that are linked to clients yet the name is in the client names table
len(Vendors[Vendors['Name'].str.lower().isin(Clients['Name'].str.lower().to_list
                                             ())==True])

735

In [187]:
from_list =Vendors2[Vendors2['CleanedAddress'].isna()==False].drop_duplicates(subset=['CleanedAddress'])['CleanedAddress'].to_list()
two_list=Clients[Clients['CleanedAddress'].isna()==False].drop_duplicates(subset=['CleanedAddress'])['CleanedAddress'].to_list()

In [188]:
from polyfuzz.models import TFIDF
from polyfuzz import PolyFuzz

tfidf = TFIDF(n_gram_range=(1, 3))
model = PolyFuzz(tfidf)
model.match(from_list, two_list)

In [189]:
model.get_matches().sort_values('Similarity',ascending=False)

,From,To,Similarity
3807,100 east pine street llc % k property manageme...,k property management llc 100 east pine street...,1.000000
3258,college west business park llc 4701 college bo...,college west business park llc 4701 college bo...,1.000000
135,national sheriffs' association 1450 duke stree...,national sheriff's association 1450 duke stree...,1.000000
3363,abaci consulting inc 101 ne circle drive grime...,abaci consulting inc 101 ne circle drive grime...,1.000000
6618,harry a yee & associates inc 4920 freeport bou...,harry a yee & associates inc 4920 freeport bou...,1.000000
...,...,...,...
7054,lee sichter llc 45024 malulani street 1 kaneoh...,fort lee school district 255 whiteman street f...,0.275791
4808,mediasnackers 21 catherine drive tongwynlais c...,roof systems consultants gwynedd office park l...,0.269400
23,home depot dept 322532906835 the lakes nv 8890...,lake country school district (wi) 1800 vettels...,0.269328
10579,pcb piezotronics inc 3425 walden avenue depew ...,electronic interiors 1043 grand avenue saint p...,0.268260


In [190]:
matchdf=model.get_matches().sort_values('Similarity',ascending=False)

In [192]:
matchdf[matchdf['Similarity']>=.6].\
    set_index('From').\
    join(Vendors2[Vendors2['CleanedAddress'].isna()==False].\
            drop_duplicates(subset=['CleanedAddress']).\
         set_index('CleanedAddress')['Vendor'],how='inner').\
    reset_index().rename(columns = {'index':'Vendor Scrubbed Address'}).\
    set_index('To').\
    join(Clients[Clients['CleanedAddress'].isna()==False].\
            drop_duplicates(subset=['CleanedAddress']).\
         set_index('CleanedAddress')['ClientID'],how='inner').\
    reset_index().rename(columns = {'index':'Client Scrubbed Address'}).\
    sort_values('Similarity',ascending=False)

,Client Scrubbed Address,Vendor Scrubbed Address,Similarity,Vendor,ClientID
1935,k property management llc 100 east pine street...,100 east pine street llc % k property manageme...,1.000000,12076,28A99EBE0FCB499BB2BC7672C071B969
2129,ljb inc 2500 newmark drive miamisburg oh 45342,ljb inc 2500 newmark drive miamisburg oh 45342,1.000000,V17604,766BF541A64545D182FA6B7C15697477
3300,structural design group 410 south 7th street l...,structural [design] group 410 south 7th street...,1.000000,V15428,4F7CA9F91A624D479967774604F7DE88
2434,national sheriff's association 1450 duke stree...,national sheriffs' association 1450 duke stree...,1.000000,10383,M_XACA001588712575000000000000
1060,college west business park llc 4701 college bo...,college west business park llc 4701 college bo...,1.000000,11838,ADVCLIENT12-300103
...,...,...,...,...,...
3740,waterloo community school district (ia) 1516 w...,courier communications po box 540 waterloo ia ...,0.600108,V10305,M_ACTA001560217038000000000000
1169,cresa washington (dc) 1800 m street nw washing...,precision systems inc 80 m street se washingto...,0.600089,V16471,3DDA17707D1041B0B742AEAF89913964
1105,complete llc 8666 west 96th street overland pa...,savage & browning 8676 west 96th street suite ...,0.600062,V14163,BA2FE2FC27364B128BE36639FBE12701
19,4 site advisors 1501 wazee street suite 1c den...,tacito design inc 1743 wazee street denver co ...,0.600039,V15285,71975B6C5C7F44A494FA131E4FB53650


In [193]:
matchdf=matchdf[matchdf['Similarity']>=.6].\
    set_index('From').\
    join(Vendors2[Vendors2['CleanedAddress'].isna()==False].\
            drop_duplicates(subset=['CleanedAddress']).\
         set_index('CleanedAddress')['Vendor'],how='inner').\
    reset_index().rename(columns = {'index':'Vendor Scrubbed Address'}).\
    set_index('To').\
    join(Clients[Clients['CleanedAddress'].isna()==False].\
            drop_duplicates(subset=['CleanedAddress']).\
         set_index('CleanedAddress')['ClientID'],how='inner').\
    reset_index().rename(columns = {'index':'Client Scrubbed Address'}).\
    sort_values('Similarity',ascending=False)

In [194]:
matchdf

,Client Scrubbed Address,Vendor Scrubbed Address,Similarity,Vendor,ClientID
1935,k property management llc 100 east pine street...,100 east pine street llc % k property manageme...,1.000000,12076,28A99EBE0FCB499BB2BC7672C071B969
2129,ljb inc 2500 newmark drive miamisburg oh 45342,ljb inc 2500 newmark drive miamisburg oh 45342,1.000000,V17604,766BF541A64545D182FA6B7C15697477
3300,structural design group 410 south 7th street l...,structural [design] group 410 south 7th street...,1.000000,V15428,4F7CA9F91A624D479967774604F7DE88
2434,national sheriff's association 1450 duke stree...,national sheriffs' association 1450 duke stree...,1.000000,10383,M_XACA001588712575000000000000
1060,college west business park llc 4701 college bo...,college west business park llc 4701 college bo...,1.000000,11838,ADVCLIENT12-300103
...,...,...,...,...,...
3740,waterloo community school district (ia) 1516 w...,courier communications po box 540 waterloo ia ...,0.600108,V10305,M_ACTA001560217038000000000000
1169,cresa washington (dc) 1800 m street nw washing...,precision systems inc 80 m street se washingto...,0.600089,V16471,3DDA17707D1041B0B742AEAF89913964
1105,complete llc 8666 west 96th street overland pa...,savage & browning 8676 west 96th street suite ...,0.600062,V14163,BA2FE2FC27364B128BE36639FBE12701
19,4 site advisors 1501 wazee street suite 1c den...,tacito design inc 1743 wazee street denver co ...,0.600039,V15285,71975B6C5C7F44A494FA131E4FB53650


In [195]:
matchdf.to_csv('C:\\Users\dmckenzie\OneDrive - DLR Group\Documents\Fuzzy Address For Vision.csv')